# ADVANCED RAG 🧠

# Load Dependencies 👩‍💻👨‍💻

Antes de ejecutar:
- Cargar un archivo .pdf que será la knowledge base
- Cargar un archivo .env con la  API KEY de OpenAI

In [ ]:
pip install pypdf langchain langchain_chroma sentence_transformers openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.7 MB/s eta 0:

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# PDF Document Loader 📚

In [ ]:
from pypdf import PdfReader

reader = PdfReader("/content/onu.pdf")
pdf_texts = [p.extract_text().strip() for p in reader.pages] # Get only text from pdf

pdf_texts = [text for text in pdf_texts if text]             # Filter empty pages

print(pdf_texts[5])

INTRODUCTION
11Nevertheless, as the present report shows, the women 
and men of the United Nations are determined in our 
efforts to address those crises and set humanity on a 
new path to peace, stability and prosperity. In the report, 
I highlight inspiring examples of our determination, 
based on our belief that a better world is not only 
necessary, but within our grasp.
Nowhere are the needs greater than in the midst of 
emergencies. 
In 2022, together with our partners on the ground, 
we coordinated humanitarian response plans for 216 
million people across 69 countries and territories 
and delivered life-saving assistance and protection 
to nearly 160 million people in urgent need. Thanks to the generosity of our donors, we mobilized a record 
$30 billion in humanitarian aid.
We responded to regional crises across the Sahel and 
the Horn of Africa, which face a deadly combination 
of drought, poverty, insecurity and the looming risk of famine. We reached 17 million people in the

❓❓ Cómo funciona PDFReader?

# Text Splitter ✂️📖

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter

character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""], # "" is splitting by char
    chunk_size=1000,
    chunk_overlap=20
)
character_split_texts = character_splitter.split_text('\n\n'.join(pdf_texts))



Y
The 2022 SDG Moment, which took place during 
the high-level week of the General Assembly, focused on the promise of inclusion, resilience and sustainability embedded in the Goals, especially in times of crisis. Convened by the Secretary-General, António Guterres, the SDG Moment is a reminder of our commitment to a better future for all, on a safe and healthy planet.
(New York; September 2022) © UN Photo/Manuel Elías OPENING BY THE  
SECRETARY-GENERAL 
HIGHLIGHTS FROM ACROSS THE 
UNITED NATIONS SYSTEM 
The wider United Nations family
Our Common Agenda
Global Crisis Response Group on Food, Energy and FinanceBlack Sea Initiative and Memorandum of Understanding 
Select transformative agendas 
Introduction

Total chunks: 289


In [ ]:
print(character_split_texts[10])
print(f"\nTotal chunks: {len(character_split_texts)}")

INTRODUCTION
11Nevertheless, as the present report shows, the women 
and men of the United Nations are determined in our 
efforts to address those crises and set humanity on a 
new path to peace, stability and prosperity. In the report, 
I highlight inspiring examples of our determination, 
based on our belief that a better world is not only 
necessary, but within our grasp.
Nowhere are the needs greater than in the midst of 
emergencies. 
In 2022, together with our partners on the ground, 
we coordinated humanitarian response plans for 216 
million people across 69 countries and territories 
and delivered life-saving assistance and protection 
to nearly 160 million people in urgent need. Thanks to the generosity of our donors, we mobilized a record 
$30 billion in humanitarian aid.
We responded to regional crises across the Sahel and 
the Horn of Africa, which face a deadly combination

Total chunks: 289


El modelo de embedding de setenceTransformers tiene una context window de 256 chars --> chunk by token

¿Cómo se hace? Se agarran los chunks splitted by character y se hace un "re-splitting" por token

In [ ]:
token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=20, tokens_per_chunk=256)

token_split_texts = []
for text in character_split_texts:
    token_split_texts += token_splitter.split_text(text)


societies their contributions and ideas. shrinking civic space, the rapid spread of mis - and disinformation and rising hate speech and misogyny are pushing people apart at a time when we need unity and solidarity more than ever. we face the highest levels of geopolitical mistrust and competition in decades. sharp divisions are growing within countries and across the global community. countries and entire regions are turning their backs on one another, leaving the urgent and essential business of jointly shaping a better future to wither on the vine. at the same time, humanity continues to grapple with balancing the great promise of technological innovations such as artificial intelligence and social media with the need to address clear threats to people ’ s rights to safety and privacy. 1 i n line with the regular budget of the united nations secretariat, the reporting period is from 1 january to 31 december 2022. given the

Total chunks: 299


In [ ]:
print(token_split_texts[10])
print(f"\nTotal chunks: {len(token_split_texts)}")

societies their contributions and ideas. shrinking civic space, the rapid spread of mis - and disinformation and rising hate speech and misogyny are pushing people apart at a time when we need unity and solidarity more than ever. we face the highest levels of geopolitical mistrust and competition in decades. sharp divisions are growing within countries and across the global community. countries and entire regions are turning their backs on one another, leaving the urgent and essential business of jointly shaping a better future to wither on the vine. at the same time, humanity continues to grapple with balancing the great promise of technological innovations such as artificial intelligence and social media with the need to address clear threats to people ’ s rights to safety and privacy. 1 i n line with the regular budget of the united nations secretariat, the reporting period is from 1 january to 31 december 2022. given the

Total chunks: 299


Notar que antes teniamos 289 chunks y ahora 299

# ChromaDB 🗃️

In [ ]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

embedding_function = SentenceTransformerEmbeddingFunction()  # Es una extension de BERT
print(embedding_function([token_split_texts[10]]))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[-0.06013256683945656, -0.011536662466824055, -0.0009850459173321724, -0.003142074914649129, 0.03382556512951851, 0.018097465857863426, -0.05405852571129799, -0.0041429451666772366, -0.016484374180436134, 4.11705377700855e-06, 0.005639712326228619, 0.0006582592613995075, 0.005302970763295889, 0.014318215660750866, -0.051750488579273224, -0.020262321457266808, -0.052479203790426254, -0.06903793662786484, -0.06831381469964981, -0.010312866419553757, -0.031692590564489365, 0.03703755512833595, 0.04876486212015152, 0.024439020082354546, -0.08330348879098892, -0.025349639356136322, -0.04280533269047737, -0.1040457934141159, -0.004375432152301073, 0.030696570873260498, 0.07498220354318619, -0.011164210736751556, 0.04723243787884712, 0.047450724989175797, -0.0012039892608299851, -0.005077369976788759, 0.07773368805646896, -0.019466325640678406, 0.09372986853122711, -0.04795854538679123, 0.021060721948742867, -0.1134086325764656, -0.018535111099481583, 0.0008672057301737368, 0.007074453402310

In [ ]:
chroma_client.list_collections()

In [ ]:
COLLECTION_NAME = "onu_demo_collection"

chroma_client = chromadb.Client()

existing_collections = chroma_client.list_collections()
existing_collection_names = [collection.name for collection in existing_collections]

if COLLECTION_NAME not in existing_collection_names:
  chroma_collection = chroma_client.create_collection(
      name=COLLECTION_NAME,
      embedding_function=embedding_function
  )
else:
    chroma_collection = chroma_client.get_collection(name=COLLECTION_NAME)

ids = [str(i) for i in range(len(token_split_texts))]

chroma_collection.add(ids=ids, documents=token_split_texts)


In [ ]:
chroma_collection.count()

299

# Retrieval 🔎

In [ ]:
def retrieve_k_similar_docs(query, k=5):

  results = chroma_collection.query(query_texts=query, n_results=k, include=['documents', 'embeddings'])
  retrieved_documents = results['documents'][0]

  list(map(lambda document: print(document, '\n'), retrieved_documents))

  return retrieved_documents, results

In [ ]:
query = "How many humanitarian response plans were coordinated in 2022?"
retrieved_documents, results = retrieve_k_similar_docs(query, k=5)

effective coordination of humanitarian assistance effective coordination of humanitarian assistancedetermined : report of the secretary - general on the work of the organization 2023 135 134 determined : report of the secretary - general on the work of the organization 2023key objectives the united nations works to ensure coordinated, coherent, effective and timely humanitarian responses to save lives and alleviate suffering in disasters, conflicts and other emergencies. with partners, we advocate for humanitarian principles, promote respect for international humanitarian and human rights law and mobilize resources to prepare for and respond to crises. early action and rapid response, including through anticipatory approaches, remain crucial to effective coordination. the united nations also brings governments, partners and communities together to reduce disaster risk and losses. key outcomes 

introduction 11nevertheless, as the present report shows, the women and men of the united na

# Generate RAG Response ✏️

In [ ]:
import openai
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

openai_client = OpenAI()

¿Cómo debería comportarse el modelo cuando recibe un prompt?

In [ ]:
def rag(query, retrieved_documents, model="gpt-4o"):
    information = "\n\n".join(retrieved_documents)    # Tell the model to use the retrieved info

    messages = [
        {
            "role": "system",
            "content": "You are a helpful expert in united nations research assistant. Your users are asking questions about information contained in 2023 annual report of the secretary general of the united nations."
            "You will be shown the user's question, and the relevant information from the report. Answer the user's question using only this information."
        },
        {"role": "user", "content": f"Question: {query}. \n Information: {information}"}
    ]

    response = openai_client.chat.completions.create(  #Send request to the openAI client
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content

In [ ]:
output = rag(query=query, retrieved_documents=retrieved_documents)

output

'In 2022, the United Nations coordinated humanitarian response plans for 216 million people across 69 countries and territories.'

# Visualize embeddings in 2D 🧐

---


Project embeddings

In [ ]:
pip install umap-learn

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3070, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2863, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_in

In [ ]:
import umap
import numpy as np
from tqdm import tqdm

embeddings = chroma_collection.get(include=['embeddings'])['embeddings']
umap_transform = umap.UMAP(random_state=0, transform_seed=0).fit(embeddings)

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [ ]:
def project_embeddings(embeddings, umap_transform):
  """
  Projects high-dimensional embeddings into a 2D space using UMAP.

  Parameters:
  embeddings (numpy.ndarray): A 2D array-like object containing the embeddings to be transformed,
                              where each row represents an embedding vector.
  umap_transform (umap.UMAP): A pre-trained UMAP model to perform the transformation.

  Returns:
  numpy.ndarray: A 2D array where each row is a 2D embedding resulting from the UMAP transformation.
  """
  umap_embeddings = np.empty((len(embeddings),2))   # Mappeamos desde la longitud de nuestros embeddings a 2D

  for i, embedding in enumerate(tqdm(embeddings)):
      umap_embeddings[i] = umap_transform.transform([embedding])

  return umap_embeddings

In [ ]:
projected_dataset_embeddings = project_embeddings(embeddings, umap_transform)

100%|██████████| 299/299 [06:25<00:00,  1.29s/it]


In [ ]:
import plotly.express as px
import pandas as pd

def plot_embeddings(projected_dataset_embeddings):
    """
    Plots 2D projected embeddings using Plotly Express.

    Parameters:
    projected_dataset_embeddings (numpy.ndarray): A 2D array containing the projected embeddings,
                                                  where each row represents a point in 2D space.
    """

    df_embeddings = pd.DataFrame(projected_dataset_embeddings, columns=['x', 'y'])

    fig = px.scatter(df_embeddings, x='x', y='y', title='Projected Embeddings')

    fig.update_layout(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        showlegend=False,
        title_x=0.5
    )

    fig.update_yaxes(scaleanchor="x", scaleratio=1)  # Maintain aspect ratio
    fig.show()


In [ ]:
plot_embeddings(projected_dataset_embeddings)

### Marquemos la query y los relevant docs

In [ ]:
import plotly.graph_objects as go
import pandas as pd

def plot_relevant_docs(projected_dataset_embeddings,
                       projected_query_embedding,
                       projected_retrieved_embeddings,
                       query, title, projected_augmented_embeddings=[]):
    """
    Plots the query, dataset, and retrieved documents in 2D embedding space using Plotly.

    Parameters:
    projected_dataset_embeddings (numpy.ndarray): 2D array of projected dataset embeddings.
    projected_query_embedding (numpy.ndarray): 2D array containing the projected query embedding.
    projected_retrieved_embeddings (numpy.ndarray): 2D array of projected embeddings for retrieved documents.
    query (str): The query text to be displayed on hover.
    title (str): Title of the plot.
    """
    # Convert data to DataFrame for easier plotting
    df_dataset = pd.DataFrame(projected_dataset_embeddings, columns=['x', 'y'])
    df_query = pd.DataFrame(projected_query_embedding, columns=['x', 'y'])
    df_query['text'] = query  # Add query text for hover information
    df_retrieved = pd.DataFrame(projected_retrieved_embeddings, columns=['x', 'y'])
    df_augmented = pd.DataFrame(projected_augmented_embeddings, columns=['x', 'y'])

    # Create a scatter plot for the dataset points
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=df_dataset['x'],
        y=df_dataset['y'],
        mode='markers',
        marker=dict(size=10, color='gray'),
        name='Dataset Embeddings'
    ))

    fig.add_trace(go.Scatter(
        x=df_query['x'],
        y=df_query['y'],
        mode='markers',
        marker=dict(size=15, symbol='x', color='red'),
        name='Query Embedding',
        hovertext=[query],  # Add hover text
        hoverinfo='text'
    ))

    fig.add_trace(go.Scatter(
        x=df_retrieved['x'],
        y=df_retrieved['y'],
        mode='markers',
        marker=dict(size=12, symbol='circle-open', line=dict(color='green', width=2)),
        name='Retrieved Embeddings'
    ))

    fig.add_trace(go.Scatter(
        x=df_augmented['x'],
        y=df_augmented['y'],
        mode='markers',
        marker=dict(size=15, symbol='x', color='pink'),
        name='Augmented Embeddings'
    ))

    fig.update_layout(
        title=title,
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        showlegend=True,
        title_x=0.5
    )

    fig.show()


In [ ]:
def project_and_plot_relevant_docs(query, title):

  retrieved_documents, results = retrieve_k_similar_docs(query, k=5)
  query_embedding = embedding_function([query])[0]
  retrieved_embeddings = results['embeddings'][0]

  projected_query_embedding = project_embeddings([query_embedding], umap_transform)
  projected_retrieved_embeddings = project_embeddings(retrieved_embeddings, umap_transform)

  plot_relevant_docs(projected_dataset_embeddings, projected_query_embedding, projected_retrieved_embeddings, query, title)
  return retrieved_documents

In [ ]:
query = "How many humanitarian response plans were coordinated in 2022?"
project_and_plot_relevant_docs(query, title=query)

effective coordination of humanitarian assistance effective coordination of humanitarian assistancedetermined : report of the secretary - general on the work of the organization 2023 135 134 determined : report of the secretary - general on the work of the organization 2023key objectives the united nations works to ensure coordinated, coherent, effective and timely humanitarian responses to save lives and alleviate suffering in disasters, conflicts and other emergencies. with partners, we advocate for humanitarian principles, promote respect for international humanitarian and human rights law and mobilize resources to prepare for and respond to crises. early action and rapid response, including through anticipatory approaches, remain crucial to effective coordination. the united nations also brings governments, partners and communities together to reduce disaster risk and losses. key outcomes 

introduction 11nevertheless, as the present report shows, the women and men of the united na

100%|██████████| 5/5 [00:09<00:00,  1.85s/it]


['effective coordination of humanitarian assistance effective coordination of humanitarian assistancedetermined : report of the secretary - general on the work of the organization 2023 135 134 determined : report of the secretary - general on the work of the organization 2023key objectives the united nations works to ensure coordinated, coherent, effective and timely humanitarian responses to save lives and alleviate suffering in disasters, conflicts and other emergencies. with partners, we advocate for humanitarian principles, promote respect for international humanitarian and human rights law and mobilize resources to prepare for and respond to crises. early action and rapid response, including through anticipatory approaches, remain crucial to effective coordination. the united nations also brings governments, partners and communities together to reduce disaster risk and losses. key outcomes',
 'introduction 11nevertheless, as the present report shows, the women and men of the unite

In [ ]:
query = "What's RAG?"
project_and_plot_relevant_docs(query, title=query)

crises and enhancing inter - agency cooperation on the prevention agenda. we also developed a new policy that formally recognizes the linkages between sexual exploitation and abuse and human rights and provides practical guidance on a human rights - based, victim - centred approach to preventing and responding to sexual exploitation and abuse. we are working closely with united nations country teams and united nations entities on its implementation. non - discrimination with our support, over 700 participants from states, civil society and the united nations took part in the inaugural session of the permanent forum on people of african descent. this event will contribute to a united nations declaration on the promotion, protection and full respect of the human rights of people of african descent. we deployed anti - racial discrimination advisers to our regional human rights offices in bangkok, beirut, brussels, pretoria and santiago to strengthen our capacity and support 

disarmament 

100%|██████████| 5/5 [00:05<00:00,  1.04s/it]


['crises and enhancing inter - agency cooperation on the prevention agenda. we also developed a new policy that formally recognizes the linkages between sexual exploitation and abuse and human rights and provides practical guidance on a human rights - based, victim - centred approach to preventing and responding to sexual exploitation and abuse. we are working closely with united nations country teams and united nations entities on its implementation. non - discrimination with our support, over 700 participants from states, civil society and the united nations took part in the inaugural session of the permanent forum on people of african descent. this event will contribute to a united nations declaration on the promotion, protection and full respect of the human rights of people of african descent. we deployed anti - racial discrimination advisers to our regional human rights offices in bangkok, beirut, brussels, pretoria and santiago to strengthen our capacity and support',
 'disarmam

# Query Expansion 🧠📝

### Generate a possible answer with another LLM module

In [ ]:
def augment_query_generated(query, model="gpt-4o"):
    messages = [
        {
            "role": "system",
            "content": "You are a helpful expert in united nations research assistant. Your users are asking questions about information contained in 2023 annual report of the secretary general of the united nations."
        },
        {"role": "user", "content": query}
    ]

    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content

In [ ]:
original_query = "How many humanitarian response plans were coordinated in 2022?"
hypothetical_answer = augment_query_generated(original_query)

joint_query = f"{original_query} {hypothetical_answer}"
joint_query

'How many humanitarian response plans were coordinated in 2022? In 2022, the United Nations coordinated 29 humanitarian response plans.'

In [ ]:
project_and_plot_relevant_docs(joint_query, title="Query Expansion")

effective coordination of humanitarian assistance effective coordination of humanitarian assistancedetermined : report of the secretary - general on the work of the organization 2023 135 134 determined : report of the secretary - general on the work of the organization 2023key objectives the united nations works to ensure coordinated, coherent, effective and timely humanitarian responses to save lives and alleviate suffering in disasters, conflicts and other emergencies. with partners, we advocate for humanitarian principles, promote respect for international humanitarian and human rights law and mobilize resources to prepare for and respond to crises. early action and rapid response, including through anticipatory approaches, remain crucial to effective coordination. the united nations also brings governments, partners and communities together to reduce disaster risk and losses. key outcomes 

introduction 11nevertheless, as the present report shows, the women and men of the united na

100%|██████████| 5/5 [00:05<00:00,  1.00s/it]


['effective coordination of humanitarian assistance effective coordination of humanitarian assistancedetermined : report of the secretary - general on the work of the organization 2023 135 134 determined : report of the secretary - general on the work of the organization 2023key objectives the united nations works to ensure coordinated, coherent, effective and timely humanitarian responses to save lives and alleviate suffering in disasters, conflicts and other emergencies. with partners, we advocate for humanitarian principles, promote respect for international humanitarian and human rights law and mobilize resources to prepare for and respond to crises. early action and rapid response, including through anticipatory approaches, remain crucial to effective coordination. the united nations also brings governments, partners and communities together to reduce disaster risk and losses. key outcomes',
 'introduction 11nevertheless, as the present report shows, the women and men of the unite

### Query expansion with multiple queries 🧠📝

---



In [ ]:
def augment_multiple_query(query, model="gpt-3.5-turbo"):
    messages = [
        {
            "role": "system",
            "content": "You are a helpful expert in united nations research assistant. Your users are asking questions about information contained in 2023 annual report of the secretary general of the united nations."
            "Suggest up to five additional related questions to help them find the information they need, for the provided question. "
            "Suggest only short questions without compound sentences. Suggest a variety of questions that cover different aspects of the topic."
            "Make sure they are complete questions, and that they are related to the original question."
            "Output one question per line. Do not number the questions."
        },
        {"role": "user", "content": query}
    ]

    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    content = content.split("\n")
    return content

In [ ]:
original_query = "How many humanitarian response plans were coordinated in 2022?"
augmented_queries = augment_multiple_query(original_query)

for query in augmented_queries: print(query)

- What were the key priorities outlined in the 2023 annual report of the Secretary-General of the United Nations?
- Which countries or regions were identified as facing the most severe humanitarian crises in the 2023 report?
- How did the United Nations address issues of peace and security in the 2023 annual report?
- What progress was reported on achieving the Sustainable Development Goals in the 2023 annual report of the Secretary-General of the United Nations?
- How did the United Nations collaborate with member states and other partners to advance global cooperation in the 2023 report?


In [ ]:
queries = [original_query] + augmented_queries # Chroma can handle multiple queries in parallel

In [ ]:
retrieved_documents, results = retrieve_k_similar_docs(queries, k=5)

# Deduplicate the retrieved documents
unique_documents = set()
for documents in retrieved_documents:
    for document in documents:
        unique_documents.add(document)


effective coordination of humanitarian assistance effective coordination of humanitarian assistancedetermined : report of the secretary - general on the work of the organization 2023 135 134 determined : report of the secretary - general on the work of the organization 2023key objectives the united nations works to ensure coordinated, coherent, effective and timely humanitarian responses to save lives and alleviate suffering in disasters, conflicts and other emergencies. with partners, we advocate for humanitarian principles, promote respect for international humanitarian and human rights law and mobilize resources to prepare for and respond to crises. early action and rapid response, including through anticipatory approaches, remain crucial to effective coordination. the united nations also brings governments, partners and communities together to reduce disaster risk and losses. key outcomes 

introduction 11nevertheless, as the present report shows, the women and men of the united na

In [ ]:
original_query_embedding = embedding_function([original_query])
augmented_query_embeddings = embedding_function(augmented_queries)

project_original_query = project_embeddings(original_query_embedding, umap_transform)
project_augmented_queries = project_embeddings(augmented_query_embeddings, umap_transform)


100%|██████████| 5/5 [00:06<00:00,  1.28s/it]


In [ ]:
result_embeddings = results['embeddings']
result_embeddings = [item for sublist in result_embeddings for item in sublist]
projected_result_embeddings = project_embeddings(result_embeddings, umap_transform)


100%|██████████| 30/30 [00:38<00:00,  1.30s/it]


In [ ]:
plot_relevant_docs(
    projected_dataset_embeddings=projected_dataset_embeddings,
    projected_query_embedding=project_original_query,
    projected_retrieved_embeddings=projected_result_embeddings,
    query=original_query,
    title="Multiple queries",
    projected_augmented_embeddings=project_augmented_queries
)

# Reranker

In [ ]:
query = "How many humanitarian response plans were coordinated in 2022?"
retrieved_documents, results = retrieve_k_similar_docs(query, k=15)

effective coordination of humanitarian assistance effective coordination of humanitarian assistancedetermined : report of the secretary - general on the work of the organization 2023 135 134 determined : report of the secretary - general on the work of the organization 2023key objectives the united nations works to ensure coordinated, coherent, effective and timely humanitarian responses to save lives and alleviate suffering in disasters, conflicts and other emergencies. with partners, we advocate for humanitarian principles, promote respect for international humanitarian and human rights law and mobilize resources to prepare for and respond to crises. early action and rapid response, including through anticipatory approaches, remain crucial to effective coordination. the united nations also brings governments, partners and communities together to reduce disaster risk and losses. key outcomes 

introduction 11nevertheless, as the present report shows, the women and men of the united na

# Reranking 🧠📑

Un Cross-Encoder re-ranker utiliza modelos de lenguaje como BERT para re-ordenar documentos recuperados en función de su relevancia con respecto a una consulta.

- Uso de BERT: El Cross-Encoder toma una consulta y un documento recuperado y los concatena en una única secuencia de entrada.
Comparación por Pares: Para cada documento recuperado, el modelo compara la consulta y el documento juntos.
- Capa de Clasificación: La secuencia concatenada pasa por BERT y una capa de clasificación final que produce un puntaje (score) que indica la relevancia del documento con respecto a la consulta.
- Re-ranking: Todos los documentos recuperados se ordenan de acuerdo con sus puntajes, re-rankeando así los documentos más relevantes más arriba en la lista.

In [ ]:
from sentence_transformers import CrossEncoder
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning:

`clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884



In [ ]:
pairs = [[query, doc] for doc in retrieved_documents]
scores = cross_encoder.predict(pairs)
print("Scores:")
for score in scores: print(score)

Scores:
3.224207
7.7809076
9.342308
2.2655816
-1.6226724
4.586841
3.1569786
3.1882439
3.6973758
2.0854092
4.3365617
-5.529007
2.9695544
-1.4291809
-2.441059


In [ ]:
print("New Ordering:")
for order in np.argsort(scores)[::-1]: print(order)

New Ordering:
2
1
5
10
8
0
7
6
12
3
9
13
4
14
11


In [ ]:
retrieved_documents[4]

'introduction introductiondetermined : report of the secretary - general on the work of the organization 2023 27 26 determined : report of the secretary - general on the work of the organization 2023in focus : global crisis response group on food, energy and finance determined : report of the secretary - general on the work of the organization 2023 27 26introduction in march 2022, the united nations established the global crisis response group to respond to the war in ukraine and its global impacts on food, energy, and finance. learn more8 june second global crisis response group brief published 12 march unctad rapid assessment published 12 march the genesis : united nations senior women leaders retreat 24 february full - scale invasion of ukraine by the russian federation22 july black sea initiative and memorandum of understanding signed june'